In [1]:
import pandas as pd
import random
import re

# tsv 파일을 DataFrame으로 읽기
user = pd.read_csv('user.tsv', sep='\t', names=['User', 'History', 'Train', 'Test'])
user_raw = pd.read_csv('user(raw).tsv', sep='\t',  names=['User', 'History', 'Train', 'Test'])

history_news = pd.read_csv('history/news.tsv', sep='\t',  names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'])

train_news = pd.read_csv('train/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'])
train_behaviors = pd.read_csv('train/behaviors.tsv', sep='\t', names=['User ID', 'Click time', 'Click history', 'click'])

test_news = pd.read_csv('test/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'])
test_behaviors = pd.read_csv('test/behaviors.tsv', sep='\t', names=['User ID', 'Click time', 'Click history', 'click'])



In [2]:
# publish 순서에 맞게 오름차순으로 정렬
history_news_sorted = history_news.sort_values(by='Publish', ascending=True).reset_index(drop=True)
history_news_sorted

,News ID,Publish,Title,Click time history,Category
0,N800,2000-11-07 15:29:05,Liker ikke harry navn,2017-01-02 21:54:18,NaN
1,N395,2001-09-10 10:01:11,50 kirkebranner påsatt,2017-01-02 02:14:53,nyheter|innenriks
2,N117,2001-12-17 13:48:43,- Ringenes Herre er ingen barnefilm,2017-01-01 09:57:07,NaN
3,N1393,2002-04-09 15:00:31,Hans Nissens gate,2017-01-04 02:32:32,nyheter|trondheim
4,N409,2002-05-24 19:33:56,Folkeliv,2017-01-02 02:37:05,NaN
...,...,...,...,...,...
1511,N1471,2017-01-03 20:30:00,Kvinner reiser utenlands for å oppfylle babydr...,"2017-01-04 06:03:34,2017-01-04 06:37:16,2017-0...",NaN
1512,N1464,2017-01-03 20:52:04,- Du skal måke fortauet utenfor huset ditt,"2017-01-04 05:54:35,2017-01-04 05:54:36,2017-0...",nyheter|trondheim
1513,N1469,2017-01-03 20:54:15,Nå blir det flere «klineseter» i kinosalen,"2017-01-04 05:59:30,2017-01-04 06:01:12,2017-0...",NaN
1514,N1490,2017-01-03 21:39:58,Ladbare hybrider på sterk fremmarsj,"2017-01-04 06:42:38,2017-01-04 06:42:41,2017-0...",nyheter|sortrondelag


In [3]:
# publish 순서에 맞게 오름차순으로 정렬
train_news_sorted = train_news.sort_values(by='Publish', ascending=True).reset_index(drop=True)
train_news_sorted

,News ID,Publish,Title,Click time history,Category
0,N2609,2001-02-16 23:00:00,Thalliumdrapene på Ranheim,2017-01-05 21:34:07,NaN
1,N2256,2001-07-10 06:39:42,Tropenatt under åpen himmel,2017-01-05 04:53:20,nyheter|trondheim
2,N2699,2001-11-05 08:21:56,Paul Fjermstads veg,2017-01-05 23:19:54,nyheter|trondheim
3,N3476,2002-05-03 06:38:09,Kriminelle vil inn i Bandidos,2017-01-07 05:37:13,nyheter|trondheim
4,N2689,2002-05-23 09:55:36,Harald Bothners veg,2017-01-05 23:16:24,nyheter|trondheim
...,...,...,...,...,...
2015,N3497,2017-01-06 20:38:25,Politiet håper obduksjonen vil gi svar på døds...,"2017-01-07 06:42:10,2017-01-07 06:48:12",nyheter|innenriks
2016,N3504,2017-01-06 21:51:55,"Fryktelig høyt, men ingen katastrofe","2017-01-07 06:57:42,2017-01-07 06:59:37,2017-0...",pluss|meninger
2017,N3519,2017-01-06 22:13:16,De flinkeste elevene må stort sett klare seg selv,"2017-01-07 07:22:31,2017-01-07 07:23:17,2017-0...",pluss|meninger
2018,N3517,2017-01-06 22:14:41,En nasjonal ulykke,"2017-01-07 07:21:04,2017-01-07 07:24:36,2017-0...",pluss|kultur


## User의 history,train news title 추출

In [42]:
ID = 'U2'

history = user[user['User']==ID]['History'].iloc[0]

# History에서 News ID 추출
news_ids = []
[news_ids.append(entry.split(',')[0]) for entry in history.split(';') if entry and entry.split(',')[0] not in news_ids]

# News ID의 맞는 Title 추출
titles = []
for news_id in news_ids:
    matching_rows = history_news[history_news['News ID'] == news_id]
    if not matching_rows.empty:
        titles.append(matching_rows.iloc[0]['Title'])
        


train = user[user['User']==ID]['Train'].iloc[0]

# Train에서 News ID 추출
train_ids = []
[train_ids.append(entry.split(',')[0]) for entry in train.split(';') if entry and entry.split(',')[0] not in train_ids]

# News ID의 맞는 Title 추출
train_titles = []
for train_id in train_ids:
    matching_rows = train_news[train_news['News ID'] == train_id]
    if not matching_rows.empty:
        train_titles.append(matching_rows.iloc[0]['Title'])


print(news_ids)
print(len(news_ids))

print(titles) 
print(len(titles))

print(train_ids) 
print(len(train_ids))

print(train_titles) 
print(len(train_titles))

['N1', 'N3', 'N98', 'N329', 'N15', 'N385', 'N817', 'N831', 'N868', 'N982', 'N979', 'N898', 'N1031', 'N785', 'N1057', 'N1092']
16
['Se lesernes nyttårsbilder', '- Det blir fyrverkeri', 'Startet det nye året med fyrverkerishow', 'Fire kjørt til sykehus', 'Hvem syns du er Årets trønder?', 'Norges landslagssjef ville ha russisk leder utestengt', 'Vil du bli sprekere i 2017? Eksperten har et klart råd', 'Intens pengejakt pågår nå for å få skiheltene til Trondheim', 'Politiet frigir navn på kvinne som døde etter E39-ulykke', '- Dette er ikke Senterpartiets olje- og gasspolitikk', 'Ola Borten Moe og Ivar Vigdenes påstår at det foregår en avkristning av Norge som rektorer, politikere, ateister og humanetikere står bak?Er det sant?', 'Sørloth ville ha sagt nei om Rosenborg kom på banen i dag', 'Kokt kujur til middag', 'Slik bor Trondheims «Anno»-deltagere', '- Helt ufattelig at det går an å behandle en ny bil på denne måten', '- Helt greit hvis bensinprisen blir 19-20 kroner']
16
['N2105', 'N21

## Negative News 추출 (History News) 

In [43]:
# news_id에서 각 news_id에 대한 유사한 publish 시간 뉴스 기사 찾기
negative_ids = []
negative_titles = []
used_ids = set(news_ids)  # 중복을 방지하기 위해 사용된 뉴스 ID를 추적

for news_id in news_ids:
    idx = history_news_sorted[history_news_sorted['News ID'] == news_id].index[0]
    above_idx, below_idx = idx - 1, idx + 1
    similar_ids = []
    similar_titles = []

    # 이미 존재하는 news_id와 중복을 제외하고 위에서 가장 가까운 두 개의 뉴스를 찾기
    above_count = 0
    while above_idx >= 0 and above_count < 2:
        if history_news_sorted.loc[above_idx, 'News ID'] not in used_ids:
            similar_ids.append(history_news_sorted.loc[above_idx, 'News ID'])
            similar_titles.append(history_news_sorted.loc[above_idx, 'Title'])
            used_ids.add(history_news_sorted.loc[above_idx, 'News ID'])
            above_count += 1
        above_idx -= 1

    # 이미 존재하는 news_id와 중복을 제외하고 아래에서 가장 가까운 두 개의 뉴스를 찾기
    below_count = 0
    while below_idx < len(history_news_sorted) and below_count < 2:
        if history_news_sorted.loc[below_idx, 'News ID'] not in used_ids:
            similar_ids.append(history_news_sorted.loc[below_idx, 'News ID'])
            similar_titles.append(history_news_sorted.loc[below_idx, 'Title'])
            used_ids.add(history_news_sorted.loc[below_idx, 'News ID'])
            below_count += 1
        below_idx += 1

    negative_ids.append(similar_ids)
    negative_titles.append(similar_titles)


print(negative_ids)  
print(negative_titles)  

history_news_sorted

[['N596', 'N8', 'N2', 'N588'], ['N653', 'N7', 'N598', 'N1430'], ['N148', 'N577', 'N1402', 'N582'], ['N333', 'N322', 'N335', 'N350'], ['N1001', 'N13', 'N566', 'N587'], ['N426', 'N408', 'N387', 'N432'], ['N783', 'N784', 'N829', 'N839'], ['N773', 'N771', 'N830', 'N837'], ['N843', 'N1151', 'N874', 'N878'], ['N981', 'N933', 'N999', 'N1006'], ['N927', 'N1008', 'N1032', 'N1040'], ['N905', 'N879', 'N902', 'N904'], ['N925', 'N914', 'N1053', 'N1066'], ['N101', 'N218', 'N943', 'N1192'], ['N910', 'N758', 'N1067', 'N1076'], ['N1097', 'N1080', 'N1098', 'N1016']]
[['Måtte i oppvaskmøte etter å ha kritisert landslaget', 'På dette bildet skiller Magnus Carlsen seg ut: - Litt tilfeldig', 'Her koker det over for Tønseth. Så stakk han fra stadion i sinne.', 'Kong Harald: - Litt uvirkelig å bli 80'], ['Russer viste monsterkrefter: - Noe av det råeste jeg har sett', 'Stina Nilsson parkerte de norske jentene på sprinten', 'Lurt av appen', 'Halvparten tror neste generasjon får det verre økonomisk'], ['Nødblus

,News ID,Publish,Title,Click time history,Category
0,N800,2000-11-07 15:29:05,Liker ikke harry navn,2017-01-02 21:54:18,NaN
1,N395,2001-09-10 10:01:11,50 kirkebranner påsatt,2017-01-02 02:14:53,nyheter|innenriks
2,N117,2001-12-17 13:48:43,- Ringenes Herre er ingen barnefilm,2017-01-01 09:57:07,NaN
3,N1393,2002-04-09 15:00:31,Hans Nissens gate,2017-01-04 02:32:32,nyheter|trondheim
4,N409,2002-05-24 19:33:56,Folkeliv,2017-01-02 02:37:05,NaN
...,...,...,...,...,...
1511,N1471,2017-01-03 20:30:00,Kvinner reiser utenlands for å oppfylle babydr...,"2017-01-04 06:03:34,2017-01-04 06:37:16,2017-0...",NaN
1512,N1464,2017-01-03 20:52:04,- Du skal måke fortauet utenfor huset ditt,"2017-01-04 05:54:35,2017-01-04 05:54:36,2017-0...",nyheter|trondheim
1513,N1469,2017-01-03 20:54:15,Nå blir det flere «klineseter» i kinosalen,"2017-01-04 05:59:30,2017-01-04 06:01:12,2017-0...",NaN
1514,N1490,2017-01-03 21:39:58,Ladbare hybrider på sterk fremmarsj,"2017-01-04 06:42:38,2017-01-04 06:42:41,2017-0...",nyheter|sortrondelag


## Negative News 추출 (Question) 

In [44]:
# news_id에서 각 news_id에 대한 유사한 publish 시간 뉴스 기사 찾기
negative_train_ids = []
negative_train_titles = []
used_ids = set(train_ids)  # 중복을 방지하기 위해 used_ids에 train_id를 추가
print(used_ids)

for train_id in train_ids:
    idx = train_news_sorted[train_news_sorted['News ID'] == train_id].index[0] 
    # print(idx)
    above_idx, below_idx = idx - 1, idx + 1
    similar_ids = []
    similar_titles = []

    # 이미 존재하는 News ID와 중복된 News를 제외하고 위에서 가장 가까운 두 개의 News를 find
    while above_idx >= 0 and len(similar_ids) < 2:
        if train_news_sorted.loc[above_idx, 'News ID'] not in used_ids:
            similar_ids.append(train_news_sorted.loc[above_idx, 'News ID'])
            similar_titles.append(train_news_sorted.loc[above_idx, 'Title'])
            used_ids.add(train_news_sorted.loc[above_idx, 'News ID'])
        above_idx -= 1

    # 이미 존재하는 News ID와 중복된 News를 제외하고 아래에서 가장 가까운 두 개의 News를 find
    while below_idx < len(train_news_sorted) and len(similar_ids) < 4:
        if train_news_sorted.loc[below_idx, 'News ID'] not in used_ids:
            similar_ids.append(train_news_sorted.loc[below_idx, 'News ID'])
            similar_titles.append(train_news_sorted.loc[below_idx, 'Title'])
            used_ids.add(train_news_sorted.loc[below_idx, 'News ID'])
        below_idx += 1

    negative_train_ids.append(similar_ids)
    negative_train_titles.append(similar_titles)

# train_ids에 대한 유사한 뉴스 ID 및 제목의 결과 목록
print(negative_train_ids)  # List of news IDs
print(negative_train_titles)  # List of titles

{'N2337', 'N1817', 'N1813', 'N3197', 'N2172', 'N2094', 'N2105'}
[['N2097', 'N2175', 'N2118', 'N2125'], ['N1572', 'N1587', 'N1535', 'N1530'], ['N2093', 'N2077', 'N2160', 'N2163'], ['N1804', 'N1808', 'N1806', 'N1845'], ['N2080', 'N1973', 'N3344', 'N2886'], ['N2333', 'N2313', 'N2387', 'N2458'], ['N3184', 'N3183', 'N3220', 'N3201']]
[['RBK-unggutter på vei bort fra Lerkendal', 'Trafikkuhell i Elgesetergate', 'Gråtende Weng ba om unnskyldning til lagvenninnen', 'Advokat klager på henleggelsen av Drevland-saken'], ['Nå kommer de nye fotoboksene', 'Fant ikke spor av gift etter dødelig dose', '- Jeg fikk svingt unna i siste liten', 'Apollo kutter ut charterturer til Tyrkia'], ['Norsk sjef tror Ustjugov kan dominere i 10 år', 'Utrolig, sann historie om bortkommen 5-åring i India', 'John Carew fronter e-post som truer kundene med gebyr: - Dette er ulovlig', 'Bob Bradley i Oslo. Bekrefter møte med NFF'], ['Disse vil bli lensmann i Meråker og Frosta', 'Ungdom får tilbake førerkort for å bli yrkess

In [45]:
train_ids

['N2105', 'N2172', 'N2094', 'N1817', 'N1813', 'N2337', 'N3197']

In [46]:
question_ids = [negative_list + [title] for negative_list, title in zip(negative_train_ids, train_ids)]
question_titles = [negative_list + [title] for negative_list, title in zip(negative_train_titles, train_titles)]

print(question_ids)
print(question_titles)
print()

hidden_positions = []

# 각 행을 섞고 히든 값의 위치를 저장
for row in question_titles:
    hidden_value = row[-1]  # 히든 값 (마지막 요소)
    random.shuffle(row)  # 전체 행을 섞기
    
    hidden_index = row.index(hidden_value)  # 히든 값의 새로운 위치 찾기
    hidden_positions.append(hidden_index+1)  # 히든 값의 위치 저장

print("Shuffled Question IDs:", question_titles)
print("Hidden Positions:", hidden_positions)

[['N2097', 'N2175', 'N2118', 'N2125', 'N2105'], ['N1572', 'N1587', 'N1535', 'N1530', 'N2172'], ['N2093', 'N2077', 'N2160', 'N2163', 'N2094'], ['N1804', 'N1808', 'N1806', 'N1845', 'N1817'], ['N2080', 'N1973', 'N3344', 'N2886', 'N1813'], ['N2333', 'N2313', 'N2387', 'N2458', 'N2337'], ['N3184', 'N3183', 'N3220', 'N3201', 'N3197']]
[['RBK-unggutter på vei bort fra Lerkendal', 'Trafikkuhell i Elgesetergate', 'Gråtende Weng ba om unnskyldning til lagvenninnen', 'Advokat klager på henleggelsen av Drevland-saken', 'Æ-en ble en nedtur'], ['Nå kommer de nye fotoboksene', 'Fant ikke spor av gift etter dødelig dose', '- Jeg fikk svingt unna i siste liten', 'Apollo kutter ut charterturer til Tyrkia', 'Rælingen - Byåsen'], ['Norsk sjef tror Ustjugov kan dominere i 10 år', 'Utrolig, sann historie om bortkommen 5-åring i India', 'John Carew fronter e-post som truer kundene med gebyr: - Dette er ulovlig', 'Bob Bradley i Oslo. Bekrefter møte med NFF', 'Iversen varslet legen tre dager før kollapsen'], ['

# Positive & Negative 

In [47]:
number = len(titles)

print("[News of interest to users]")
for i in range(number):
    # 제목과 부정적인 제목들을 모두 합쳐서 셔플
    combined_titles = [titles[i]] + negative_titles[i]
    random.shuffle(combined_titles)
    
    # 마지막 요소를 제외하고 "/"로 연결된 문자열 출력
    print(f"{i+1}) ", end="")
    for j in range(len(combined_titles) - 1):
        print(f"{combined_titles[j]} / ", end="")
    # 마지막 요소 출력
    print(f"{combined_titles[-1]}")
    
    print(f"Of the five news above, the news that the user is most interested in : {titles[i]}\n")
    
print("[Questions]")
print("Rank the five candidates for each question based on the user's interest in previously read articles.")

for i, titles in enumerate(question_titles):
    print(f"Question {i+1}) ", end="")
    for j, title in enumerate(titles): 
        # print(f"{j+1}: {title} / ", end="")
        if j == len(titles) - 1:
            print(f"{j+1}: {title}", end="")
        else:
            print(f"{j+1}: {title} / ", end="")
    print()

print("\nDo not explain reasons in the response, just return a list of numbers for each article.")


[News of interest to users]
1) Måtte i oppvaskmøte etter å ha kritisert landslaget / Her koker det over for Tønseth. Så stakk han fra stadion i sinne. / Kong Harald: - Litt uvirkelig å bli 80 / På dette bildet skiller Magnus Carlsen seg ut: - Litt tilfeldig / Se lesernes nyttårsbilder
Of the five news above, the news that the user is most interested in : Se lesernes nyttårsbilder

2) Lurt av appen / - Det blir fyrverkeri / Halvparten tror neste generasjon får det verre økonomisk / Russer viste monsterkrefter: - Noe av det råeste jeg har sett / Stina Nilsson parkerte de norske jentene på sprinten
Of the five news above, the news that the user is most interested in : - Det blir fyrverkeri

3) Startet det nye året med fyrverkerishow / Mann hang fast etter bil i Verdal / Døddrukne ungdommer, trusler, vold og skadeverk / 16-åring skadd i ulykke med fyrverkeri / Nødbluss sendt gjennom vindu startet branntilløp
Of the five news above, the news that the user is most interested in : Startet det

In [26]:
# 정답
print("Hidden Positions:", hidden_positions)

Hidden Positions: [1, 4, 5, 1, 1, 5, 4]


# 한번에 출력

In [49]:
ID = 'U2'

history = user[user['User']==ID]['History'].iloc[0]

# History에서 News ID 추출
news_ids = []
[news_ids.append(entry.split(',')[0]) for entry in history.split(';') if entry and entry.split(',')[0] not in news_ids]

# News ID의 맞는 Title 추출
titles = []
for news_id in news_ids:
    matching_rows = history_news[history_news['News ID'] == news_id]
    if not matching_rows.empty:
        titles.append(matching_rows.iloc[0]['Title'])
        


train = user[user['User']==ID]['Train'].iloc[0]

# Train에서 News ID 추출
train_ids = []
[train_ids.append(entry.split(',')[0]) for entry in train.split(';') if entry and entry.split(',')[0] not in train_ids]

# News ID의 맞는 Title 추출
train_titles = []
for train_id in train_ids:
    matching_rows = train_news[train_news['News ID'] == train_id]
    if not matching_rows.empty:
        train_titles.append(matching_rows.iloc[0]['Title'])


# news_id에서 각 news_id에 대한 유사한 publish 시간 뉴스 기사 찾기
negative_ids = []
negative_titles = []
used_ids = set(news_ids)  # 중복을 방지하기 위해 사용된 뉴스 ID를 추적

for news_id in news_ids:
    idx = history_news_sorted[history_news_sorted['News ID'] == news_id].index[0]
    above_idx, below_idx = idx - 1, idx + 1
    similar_ids = []
    similar_titles = []

    # 이미 존재하는 news_id와 중복을 제외하고 위에서 가장 가까운 두 개의 뉴스를 찾기
    above_count = 0
    while above_idx >= 0 and above_count < 2:
        if history_news_sorted.loc[above_idx, 'News ID'] not in used_ids:
            similar_ids.append(history_news_sorted.loc[above_idx, 'News ID'])
            similar_titles.append(history_news_sorted.loc[above_idx, 'Title'])
            used_ids.add(history_news_sorted.loc[above_idx, 'News ID'])
            above_count += 1
        above_idx -= 1

    # 이미 존재하는 news_id와 중복을 제외하고 아래에서 가장 가까운 두 개의 뉴스를 찾기
    below_count = 0
    while below_idx < len(history_news_sorted) and below_count < 2:
        if history_news_sorted.loc[below_idx, 'News ID'] not in used_ids:
            similar_ids.append(history_news_sorted.loc[below_idx, 'News ID'])
            similar_titles.append(history_news_sorted.loc[below_idx, 'Title'])
            used_ids.add(history_news_sorted.loc[below_idx, 'News ID'])
            below_count += 1
        below_idx += 1

    negative_ids.append(similar_ids)
    negative_titles.append(similar_titles)


# news_id에서 각 news_id에 대한 유사한 publish 시간 뉴스 기사 찾기
negative_train_ids = []
negative_train_titles = []
used_ids = set(train_ids)  # 중복을 방지하기 위해 used_ids에 train_id를 추가

for train_id in train_ids:
    idx = train_news_sorted[train_news_sorted['News ID'] == train_id].index[0] 
    # print(idx)
    above_idx, below_idx = idx - 1, idx + 1
    similar_ids = []
    similar_titles = []

    # 이미 존재하는 News ID와 중복된 News를 제외하고 위에서 가장 가까운 두 개의 News를 find
    while above_idx >= 0 and len(similar_ids) < 2:
        if train_news_sorted.loc[above_idx, 'News ID'] not in used_ids:
            similar_ids.append(train_news_sorted.loc[above_idx, 'News ID'])
            similar_titles.append(train_news_sorted.loc[above_idx, 'Title'])
            used_ids.add(train_news_sorted.loc[above_idx, 'News ID'])
        above_idx -= 1

    # 이미 존재하는 News ID와 중복된 News를 제외하고 아래에서 가장 가까운 두 개의 News를 find
    while below_idx < len(train_news_sorted) and len(similar_ids) < 4:
        if train_news_sorted.loc[below_idx, 'News ID'] not in used_ids:
            similar_ids.append(train_news_sorted.loc[below_idx, 'News ID'])
            similar_titles.append(train_news_sorted.loc[below_idx, 'Title'])
            used_ids.add(train_news_sorted.loc[below_idx, 'News ID'])
        below_idx += 1

    negative_train_ids.append(similar_ids)
    negative_train_titles.append(similar_titles)


question_ids = [negative_list + [title] for negative_list, title in zip(negative_train_ids, train_ids)]
question_titles = [negative_list + [title] for negative_list, title in zip(negative_train_titles, train_titles)]

hidden_positions = []

# 각 행을 섞고 히든 값의 위치를 저장
for row in question_titles:
    hidden_value = row[-1]  # 히든 값 (마지막 요소)
    random.shuffle(row)  # 전체 행을 섞기
    
    hidden_index = row.index(hidden_value)  # 히든 값의 새로운 위치 찾기
    hidden_positions.append(hidden_index+1)  # 히든 값의 위치 저장


number = len(titles)

print("[News of interest to users]")
for i in range(number):
    # 제목과 부정적인 제목들을 모두 합쳐서 셔플
    combined_titles = [titles[i]] + negative_titles[i]
    random.shuffle(combined_titles)
    
    # 마지막 요소를 제외하고 "/"로 연결된 문자열 출력
    print(f"{i+1}) ", end="")
    for j in range(len(combined_titles) - 1):
        print(f"{combined_titles[j]} / ", end="")
    # 마지막 요소 출력
    print(f"{combined_titles[-1]}")
    
    print(f"Of the five news above, the news that the user is most interested in : {titles[i]}\n")
    
print()
print("Rank the five candidates for each question based on the user's interest in previously read articles.")

for i, titles in enumerate(question_titles):
    print(f"Question {i+1}) ", end="")
    for j, title in enumerate(titles):
        # print(f"{j+1}: {title} / ", end="")
        if j == len(titles) - 1:
            print(f"{j+1}: {title}", end="")
        else:
            print(f"{j+1}: {title} / ", end="")
    print()

print("\nDo not explain reasons in the response, just return a list of numbers for each article.")


[News of interest to users]
1) Her koker det over for Tønseth. Så stakk han fra stadion i sinne. / Kong Harald: - Litt uvirkelig å bli 80 / Se lesernes nyttårsbilder / På dette bildet skiller Magnus Carlsen seg ut: - Litt tilfeldig / Måtte i oppvaskmøte etter å ha kritisert landslaget
Of the five news above, the news that the user is most interested in : Se lesernes nyttårsbilder

2) Russer viste monsterkrefter: - Noe av det råeste jeg har sett / - Det blir fyrverkeri / Stina Nilsson parkerte de norske jentene på sprinten / Lurt av appen / Halvparten tror neste generasjon får det verre økonomisk
Of the five news above, the news that the user is most interested in : - Det blir fyrverkeri

3) Nødbluss sendt gjennom vindu startet branntilløp / Startet det nye året med fyrverkerishow / 16-åring skadd i ulykke med fyrverkeri / Mann hang fast etter bil i Verdal / Døddrukne ungdommer, trusler, vold og skadeverk
Of the five news above, the news that the user is most interested in : Startet det

In [119]:
questions = input("질문을 입력하세요: ")

# 질문에서 숫자를 추출
def extract_numbers(questions):
    # pattern = r"Question \d+ : ([\d, ]+)"
    pattern = r"Question \d+: ([\d, ]+)"
    return [list(map(int, num.split(', '))) for num in re.findall(pattern, questions)]

numbers_list = extract_numbers(questions)

correct_count = 0
total_diff = 0

diff_count = 0
# 각 질문의 숫자 목록에서 hidden_positions 값의 위치 찾기
for i, (hidden, numbers) in enumerate(zip(hidden_positions, numbers_list)):
    if numbers[0] == hidden:
        print(f"Question {i + 1} : Correct")
        correct_count += 1
    else:
        position = numbers.index(hidden)
        total_diff += abs(position)
        print(f"Question {i + 1} : {position + 1}")
    diff_count += 1

# 결과 요약 출력
print(f"\n총 Correct : {correct_count}/{len(hidden_positions)}개 ({correct_count / len(hidden_positions):.3%})")
if diff_count > 0:
    print(f"평균 오차범위 : {total_diff / diff_count:.3f}")


# #####3
# print("\n##########################################\n")
# # 정답
# for i, label in enumerate(hidden_positions):
#     print(f"Question Answer {i+1} : {label}")

Question 1 : Correct
Question 2 : Correct
Question 3 : 3
Question 4 : 2
Question 5 : Correct

총 Correct : 3/5개 (60.000%)
평균 오차범위 : 0.600


In [116]:
# 정답
for i, label in enumerate(hidden_positions):
    print(f"Question Answer {i+1} : {label}")

Question Answer 1 : 5
Question Answer 2 : 5
Question Answer 3 : 4
Question Answer 4 : 4
Question Answer 5 : 3
Question Answer 6 : 5
Question Answer 7 : 2
Question Answer 8 : 5
Question Answer 9 : 1
Question Answer 10 : 3
Question Answer 11 : 2
Question Answer 12 : 1
Question Answer 13 : 5
Question Answer 14 : 4
Question Answer 15 : 5
Question Answer 16 : 1
Question Answer 17 : 1
Question Answer 18 : 1


## Ranking 조정

In [131]:
# 입력 데이터를 문자열로 정의
input_data = """
Question 1 : Correct
Question 2 : 1
Question 3 : 3
Question 4 : 3
Question 5 : 2
Question 6 : 4
Question 7 : 4
Question 8 : 1
Question 9 : Correct
Question 10 : Correct
Question 11 : 2
Question 12 : 4
Question 13 : 4
Question 14 : 2
Question 15 : 4
Question 16 : 2
Question 17 : Correct
Question 18 : 1
Question 19 : 3
Question 20 : 2
"""

# 결과 저장 리스트
result = []

# 입력 데이터를 줄 단위로 처리
for line in input_data.split('\n'):
    # ':'로 줄을 분리
    parts = line.split(' : ', 1)  # 1번만 분리해서 예외 방지

    # parts의 길이가 2이고 두 번째 부분이 숫자인지 확인
    if len(parts) == 2 and parts[1].strip().isdigit():
        new_value = int(parts[1].strip()) + 1
        result.append(f"{parts[0]} : {new_value}")
    else:  # 숫자가 없거나 다른 텍스트일 경우 그대로 추가
        result.append(line)

# 결과 출력
output = '\n'.join(result)
print(output)



Question 1 : Correct
Question 2 : 2
Question 3 : 4
Question 4 : 4
Question 5 : 3
Question 6 : 5
Question 7 : 5
Question 8 : 2
Question 9 : Correct
Question 10 : Correct
Question 11 : 3
Question 12 : 5
Question 13 : 5
Question 14 : 3
Question 15 : 5
Question 16 : 3
Question 17 : Correct
Question 18 : 2
Question 19 : 4
Question 20 : 3

